In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [3]:
from my_utils import Dictionary

n_unique = 10

src_dict = Dictionary(['<EOS>'])
tgt_dict = Dictionary(['<BOS>', '<EOS>'])
for n in range(n_unique):
    src_dict.add_word(str(n))
    tgt_dict.add_word(str(n))

In [11]:
from my_utils.toy_data import invert_seq
train = invert_seq(1000, n_unique=n_unique)
test = invert_seq(100, n_unique=n_unique)

In [12]:
import torch
from my_utils import DataLoader
from torch_models.utils import seq2seq

def numericalize(dataset, src_dict, tgt_dict):
    numericalized = [([src_dict(s) for s in src], [tgt_dict(t) for t in tgt]) for src, tgt in dataset]
    return numericalized

# device = 'cuda:0'
device = torch.device('cpu')
trans_func = seq2seq(device)

train_loader = DataLoader(numericalize(train, src_dict, tgt_dict), batch_size=64, trans_func=trans_func)
test_loader = DataLoader(numericalize(test, src_dict, tgt_dict), batch_size=50, trans_func=trans_func)

In [13]:
from torch_models.models.transformer import *
embed_size = 24
n_head = 4

model = Transformer(size=embed_size, n_head=n_head, src_vocab_size=len(src_dict), tgt_vocab_size=len(tgt_dict),
                    src_EOS=src_dict('<EOS>'), tgt_BOS=tgt_dict('<BOS>'), tgt_EOS=tgt_dict('<EOS>'),
                    dropout=0, n_layers=2)
print(model)

Transformer(
  (encoder): TransformerEncoder(
    (embedding): TransformerEmbedding(
      (embedding): Embedding(12, 24, padding_idx=11)
    )
    (pe): PositionalEncoding(
      (dropout): Dropout(p=0)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (attention): MultiHeadedAttention(
          (Q_linear): Linear(in_features=24, out_features=24, bias=True)
          (K_linear): Linear(in_features=24, out_features=24, bias=True)
          (V_linear): Linear(in_features=24, out_features=24, bias=True)
          (out_linear): Linear(in_features=24, out_features=24, bias=True)
          (attention): DotAttn()
          (dropout): Dropout(p=0)
        )
        (fc): PositionwiseFeedForward(
          (w_1): Linear(in_features=24, out_features=96, bias=True)
          (w_2): Linear(in_features=96, out_features=24, bias=True)
          (dropout): Dropout(p=0)
        )
        (layer_norms): ModuleList(
          (0): LayerNorm(torch.Size([24]), eps=1e-05, elementwise_affin

In [14]:
%%time
from my_utils import Trainer, EvaluatorSeq
from torch.optim import Adam, SGD

optimizer = Adam(model.parameters())

evaluator = EvaluatorSeq(model, test_loader, measure='BLEU')

trainer = Trainer(model, train_loader)
trainer.train(optimizer, max_epoch=5,
              evaluator=evaluator, score_monitor=None, show_log=True)

/Users/linghan/anaconda/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


epoch 0  	loss: 2.449956715106964	BLEU: 0.26368937080638133	


/Users/linghan/anaconda/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


epoch 1  	loss: 2.007535107433796	BLEU: 0.15414364092029126	
epoch 2  	loss: 1.6938202679157257	BLEU: 0.2814006513563015	
epoch 3  	loss: 1.4096672162413597	BLEU: 0.38654990661598615	
epoch 4  	loss: 1.164906620979309	BLEU: 0.13570377732484512	
CPU times: user 56.4 s, sys: 2.8 s, total: 59.2 s
Wall time: 28.6 s


In [10]:
iter(train_loader)
l = 10
inputs, targets = next(train_loader)
inputs = inputs[:l]
targets = targets[:l]
generated = model.predict(inputs)
print('======= input ======')
for seq in inputs:
    print([src_dict[s.item()] for s in seq])
print('======= output ======')
for seq in generated[:l]:
    print([tgt_dict[s] for s in seq])

======= input ======
['7', '2', '6', '1', '6']
['2', '3', '5', '9']
['9', '9', '0', '9', '3']
['3', '7', '2', '5', '8']
['5', '9', '5', '7', '9']
['9', '9', '6', '0']
['9', '1', '2', '1', '8']
['4', '8', '8']
['6', '4', '9']
['9', '7', '3', '6']
======= output ======
['6', '1', '6', '2', '7']
['9', '5', '3', '2']
['3', '9', '0', '9', '9']
['8', '5', '2', '7', '3']
['9', '7', '5', '9', '5']
['0', '6', '9', '9']
['8', '1', '2', '1', '9']
['8', '8', '4']
['9', '4', '6']
['6', '3', '7', '9']
